In [1]:
%load_ext autoreload
%autoreload 2

import sys
from tqdm.notebook import tqdm
from time import sleep
sys.path.append('../')
from src.commits import *

In [2]:
project = 'nova'

In [3]:
repo_path = f"D:\GitHubs\openstack-{project}"

In [4]:
gr = GitRepository(repo_path)

In [5]:
date_start = datetime(2011, 11, 1)
date_end = datetime(2019, 6, 30)

In [6]:
all_commit_info = {}

In [7]:
num_commits = 0

for commit in RepositoryMining(repo_path, since = date_start, to = date_end, only_in_branch = 'master').traverse_commits():
    num_commits += 1
    
print(num_commits)

43313


In [8]:
num_commits_omitted = 0

with tqdm(total = num_commits) as pbar:
    for commit in RepositoryMining(repo_path, since = date_start, to = date_end, only_in_branch = 'master').traverse_commits():
        pbar.set_postfix({'hash' : commit.hash})
        if is_valid_commit(commit):
            all_commit_info[commit.hash] = CommitInfo(commit)
        else:
            num_commits_omitted += 1
        # Progress bar
        pbar.update(1)
        sleep(0.001)

print("{:d} commits ommitted".format(num_commits_omitted))

  0%|          | 0/43313 [00:00<?, ?it/s]

21517 commits ommitted


Update prior and future info

In [11]:
with tqdm(total = len(all_commit_info)) as pbar:
    for i, (commit_hash, commit_info) in enumerate(all_commit_info.items()):
        pbar.set_postfix({'hash' : commit_hash})
        updated = update_prior_and_future_info(commit_hash, gr, all_commit_info)
        # Progress bar
        pbar.update(1)
        sleep(0.001)

  0%|          | 0/2942 [00:00<?, ?it/s]

Convert to DataFrame

In [27]:
all_commit_info_values = []
with tqdm(total = len(all_commit_info)) as pbar:
    for commit_hash, commit_info in all_commit_info.items():
        pbar.set_postfix({'hash' : commit_hash})
        all_commit_info_values.append(commit_info.toDict())
        # Progress bar
        pbar.update(1)
        sleep(0.001)

  0%|          | 0/3977 [00:00<?, ?it/s]

In [28]:
df = pd.DataFrame(data = all_commit_info_values)

Save to CSV

In [37]:
df.to_csv(f"..\data\commits\commits_{project}.csv", index = False)